# Download and load library

In [ ]:
!pip install google-play-scraper
!pip install googletrans
!pip install textblob
!pip install wordcloud
!pip install nltk
!pip install pyspellchecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15717 sha256=842fc859af841c9e17e52d18839e7e269b52df874de598d0d61492a5dfdef384
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: chardet
 

ERROR: Operation cancelled by user


In [3]:
!pip freeze > requirements.txt

In [2]:
import pandas as pd
import nltk
import re
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from google_play_scraper import app, reviews, Sort, reviews_all
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from google.colab import drive
from tensorflow.keras.callbacks import TensorBoard
import time
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import TensorBoard
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from google.colab import drive
from datetime import datetime
import time
from wordcloud import WordCloud
from tensorflow.keras.layers import TextVectorization

drive.mount('/content/drive')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


Mounted at /content/drive


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

#Scrap data

In [4]:
def main():
    countries = ['us', 'in', 'jp', 'de', 'fr', 'id', 'cn', 'tw', 'th', 'ms', 'sg', 'ph']  # List of country codes
    all_reviews = []

    for country in countries:
        scrapreview_gi = reviews_all(
            'com.miHoYo.GenshinImpact',
            lang='en',
            sort=Sort.MOST_RELEVANT,
            count=10000,
            country=country
        )
        reviews_gi = pd.DataFrame(scrapreview_gi)
        reviews_gi['country'] = country
        all_reviews.append(reviews_gi)

    combined_reviews = pd.concat(all_reviews, ignore_index=True)
    combined_reviews['at'] = pd.to_datetime(combined_reviews['at'])
    combined_reviews['repliedAt'] = pd.to_datetime(combined_reviews['repliedAt'])
    combined_reviews.to_csv('reviews_gi_combined.csv', index=False)

    return print('Selesai scrapping')
start_time = time.time()
main()
print(f"Elapsed time: {time.time() - start_time}s")

Selesai scrapping
Elapsed time: 1471.9555468559265s


In [5]:
gi = pd.read_csv('reviews_gi_combined.csv')
gi

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,country
0,8b3c2523-a5fb-4032-a162-ac0a9136e844,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"Initially, I had a 5-star review. I can no lon...",3,437,4.7.0_23093867_23435602,2024-06-10 07:46:00,NaN,NaN,4.7.0_23093867_23435602,us
1,91224269-9457-4266-b0fb-4b36c368127d,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,This is a really great game and I do enjoy it ...,3,190,4.7.0_23093867_23435602,2024-06-27 01:41:03,NaN,NaN,4.7.0_23093867_23435602,us
2,e98d378d-0e5a-46ab-b965-52c1ee878215,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"You've seen the adds, youtubers sponsoring thi...",5,1095,4.7.0_23093867_23435602,2024-06-24 14:17:51,NaN,NaN,4.7.0_23093867_23435602,us
3,2f3e1118-0407-4a05-af27-0c651ee1e1d2,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,For years now we have begged for a skip dialog...,1,3321,4.7.0_23093867_23435602,2024-07-03 05:09:53,NaN,NaN,4.7.0_23093867_23435602,us
4,34d55b58-8517-4a31-b86c-5c96c97ca9e9,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Edit: I used to love this game! It has beautif...,2,1004,4.6.0_22174900_22378599,2024-04-27 17:21:14,NaN,NaN,4.6.0_22174900_22378599,us
...,...,...,...,...,...,...,...,...,...,...,...,...
1376995,bfbac0a3-a46c-4785-846a-20418819c988,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,The life in the game is about farming most of ...,2,0,NaN,2020-12-25 03:26:55,NaN,NaN,NaN,ph
1376996,338121fc-b670-4ba0-ac70-57620aefad83,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"Cross platform game, with so much content",5,0,1.0.0_1112729_1135452,2020-10-06 08:00:32,NaN,NaN,1.0.0_1112729_1135452,ph
1376997,236918e0-f306-4c83-8c96-fb6e1fcceace,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,The game higher ups have zero respect and resp...,1,2,NaN,2021-09-29 03:34:07,NaN,NaN,NaN,ph
1376998,e0346ab8-90ed-41a8-b579-04eb25987825,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"Network error on mobile, can play fine on PC",4,0,NaN,2021-06-09 02:33:55,NaN,NaN,NaN,ph


In [6]:
gi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1377000 entries, 0 to 1376999
Data columns (total 12 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   reviewId              1377000 non-null  object
 1   userName              1377000 non-null  object
 2   userImage             1377000 non-null  object
 3   content               1377000 non-null  object
 4   score                 1377000 non-null  int64 
 5   thumbsUpCount         1377000 non-null  int64 
 6   reviewCreatedVersion  1039473 non-null  object
 7   at                    1377000 non-null  object
 8   replyContent          176190 non-null   object
 9   repliedAt             176190 non-null   object
 10  appVersion            1039473 non-null  object
 11  country               1377000 non-null  object
dtypes: int64(2), object(10)
memory usage: 126.1+ MB


In [7]:
gi.isna().sum()

reviewId                      0
userName                      0
userImage                     0
content                       0
score                         0
thumbsUpCount                 0
reviewCreatedVersion     337527
at                            0
replyContent            1200810
repliedAt               1200810
appVersion               337527
country                       0
dtype: int64

In [8]:
# new_gi = gi.sample(frac=0., random_state=42)
new_gi = gi

In [9]:
print(f'Info sample data:\n{new_gi.info()}\n')
print(f'Null values:\n{new_gi.isna().sum()}\n')
print(f'Duplicates:\n{new_gi.duplicated().sum()}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1377000 entries, 0 to 1376999
Data columns (total 12 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   reviewId              1377000 non-null  object
 1   userName              1377000 non-null  object
 2   userImage             1377000 non-null  object
 3   content               1377000 non-null  object
 4   score                 1377000 non-null  int64 
 5   thumbsUpCount         1377000 non-null  int64 
 6   reviewCreatedVersion  1039473 non-null  object
 7   at                    1377000 non-null  object
 8   replyContent          176190 non-null   object
 9   repliedAt             176190 non-null   object
 10  appVersion            1039473 non-null  object
 11  country               1377000 non-null  object
dtypes: int64(2), object(10)
memory usage: 126.1+ MB
Info sample data:
None

Null values:
reviewId                      0
userName                      0
userI

In [10]:
#change some column to datetime
new_gi['at'] = pd.to_datetime(new_gi['at'])
new_gi['repliedAt'] = pd.to_datetime(new_gi['repliedAt'])
new_gi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1377000 entries, 0 to 1376999
Data columns (total 12 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   reviewId              1377000 non-null  object        
 1   userName              1377000 non-null  object        
 2   userImage             1377000 non-null  object        
 3   content               1377000 non-null  object        
 4   score                 1377000 non-null  int64         
 5   thumbsUpCount         1377000 non-null  int64         
 6   reviewCreatedVersion  1039473 non-null  object        
 7   at                    1377000 non-null  datetime64[ns]
 8   replyContent          176190 non-null   object        
 9   repliedAt             176190 non-null   datetime64[ns]
 10  appVersion            1039473 non-null  object        
 11  country               1377000 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(8)


In [11]:
new_gi.to_csv('reviews_gi_combined.csv', index=False)#simpan ke csv

In [ ]:
# !cp "/content/reviews_gi_combined.csv" "/content/drive/My Drive/" ##uncomment jika ingin menyimpannya ke google drive

# Feature selection kolom table

In [4]:
#download dataset dari google drive
!gdown 1kazy1XBEH8Lrp3ND6nt7NBytBFmytD_O

Downloading...
From (original): https://drive.google.com/uc?id=1kazy1XBEH8Lrp3ND6nt7NBytBFmytD_O
From (redirected): https://drive.google.com/uc?id=1kazy1XBEH8Lrp3ND6nt7NBytBFmytD_O&confirm=t&uuid=770917a8-dce8-46f4-89c1-6c5c3177ee0b
To: /content/reviews_gi_combined.csv
100% 755M/755M [00:09<00:00, 79.6MB/s]


In [6]:
# new_gi = pd.read_csv("/content/drive/My Drive/reviews_gi_combined.csv")#load data dari google drive
new_gi = pd.read_csv("reviews_gi_combined.csv")
new_gi

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,country
0,8b3c2523-a5fb-4032-a162-ac0a9136e844,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"Initially, I had a 5-star review. I can no lon...",3,405,4.7.0_23093867_23435602,2024-06-10 07:46:00,NaN,NaN,4.7.0_23093867_23435602,us
1,91224269-9457-4266-b0fb-4b36c368127d,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,This is a really great game and I do enjoy it ...,3,180,4.7.0_23093867_23435602,2024-06-27 01:41:03,NaN,NaN,4.7.0_23093867_23435602,us
2,e98d378d-0e5a-46ab-b965-52c1ee878215,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"You've seen the adds, youtubers sponsoring thi...",5,1043,4.7.0_23093867_23435602,2024-06-24 14:17:51,NaN,NaN,4.7.0_23093867_23435602,us
3,2f3e1118-0407-4a05-af27-0c651ee1e1d2,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,For years now we have begged for a skip dialog...,1,2897,4.7.0_23093867_23435602,2024-07-03 05:09:53,NaN,NaN,4.7.0_23093867_23435602,us
4,34d55b58-8517-4a31-b86c-5c96c97ca9e9,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Edit: I used to love this game! It has beautif...,2,987,4.6.0_22174900_22378599,2024-04-27 17:21:14,NaN,NaN,4.6.0_22174900_22378599,us
...,...,...,...,...,...,...,...,...,...,...,...,...
1610995,147b5378-f562-4fb2-a87a-0f9a81cbbcb4,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Very good the only thing I don't like are the ...,5,0,3.7.0_14806628_14807925,2023-07-03 17:08:52,NaN,NaN,3.7.0_14806628_14807925,ph
1610996,5d9d2fd3-83d0-4e7c-b37e-2d26969e5595,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"The game is great overall, but the Company jus...",1,1,2.1.0_3981765_4073622,2021-09-28 15:13:59,NaN,NaN,2.1.0_3981765_4073622,ph
1610997,cf251f90-6308-4830-add1-a29434ebf467,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,My favorite game when I'm bored it helped me a...,5,0,NaN,2022-01-05 22:14:08,NaN,NaN,NaN,ph
1610998,236ec485-97eb-47c4-87a5-d4938d63590e,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"This is so much fun, annoying at times, but st...",5,0,1.1.1_1437351_1398019,2020-12-05 04:34:58,NaN,NaN,1.1.1_1437351_1398019,ph


In [7]:
#drop kolom tidak diperlukan
new_gi = new_gi.drop(['appVersion', 'userImage', 'userName','reviewId', 'reviewCreatedVersion', 'at', 'repliedAt', 'replyContent', 'country'], axis=1)
new_gi

,content,score,thumbsUpCount
0,"Initially, I had a 5-star review. I can no lon...",3,405
1,This is a really great game and I do enjoy it ...,3,180
2,"You've seen the adds, youtubers sponsoring thi...",5,1043
3,For years now we have begged for a skip dialog...,1,2897
4,Edit: I used to love this game! It has beautif...,2,987
...,...,...,...
1610995,Very good the only thing I don't like are the ...,5,0
1610996,"The game is great overall, but the Company jus...",1,1
1610997,My favorite game when I'm bored it helped me a...,5,0
1610998,"This is so much fun, annoying at times, but st...",5,0


#Slangs Word Dictionary

In [8]:
#for preprocessing slangs word

abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk",
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cuz" : "because",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "coz" : "because",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart",
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "im" : "i am",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "pls" : "please",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet",
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously",
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

# Fungsi Preprocessing

In [9]:
#preprocessing function

def replace_emoji(text):
  if not isinstance(text, str):
      return text
  emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002500-\U00002BEF"
        "\U00002702-\U000027B0"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "\U0001f926-\U0001f937"
        "\U00010000-\U0010ffff"
        "\u2640-\u2642"
        "\u2600-\u2B55"
        "\u200d"
        "\u23cf"
        "\u23e9"
        "\u231a"
        "\ufe0f"
        "\u3030"
        "]+", flags=re.UNICODE
    )
  return emoji_pattern.sub(' ', text)

def cleantext(text):
    text = str(text).lower()
    text = replace_emoji(text)
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r'https?:\/\/\S+', '', text)
    text = re.sub(r'(.)\1{2,}', r'\1', text)#double letters
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          str(text))
    # emoji_pattern = emoji_data.keys()
    # text = emoji_pattern.sub(r'', text)
    text = text.replace('\n', ' ')

    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip(' ')
    return text

def filteringsText(text):
    liststopword = set(stopwords.words('english'))
    filtered = []

    for word in text:
        if word not in liststopword:
            filtered.append(word)
    text = filtered

    return filtered

def tokenization(text):
    list_words = word_tokenize(str(text))
    return list_words

def toKata(list_words):
    sentence = ' '.join(word for word in list_words)
    return sentence

def abbrev(text):
    kata_kata = text.split()
    kata_fixed = []
    for kata in kata_kata:
        if kata.lower() in abbreviations:
            kata_fixed.append(abbreviations[kata.lower()])
        else:
            kata_fixed.append(kata)
    kata_fixed = ' '.join(kata_fixed)
    return kata_fixed

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    text = " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])
    return text

# df["text_lemmatized"] = df["text"].apply(lambda text: lemmatize_words(text))
# df.head()
spell = SpellChecker()

def correct_spellings(text):
    if not isinstance(text, str):
        return text
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        corrected_text.append(spell.correction(word) if word in misspelled_words else word)
    return " ".join(corrected_text)
# def lengths(text):
#     return len(text)


# Fungsi Labelling Data

In [10]:
from textblob import TextBlob
np.random.seed(42)
def subjektivitas(tr_text):
    return TextBlob(tr_text).sentiment.subjectivity

def polaritas(tr_text):
    return TextBlob(tr_text).sentiment.polarity

def hasilSentimen(nilai):
    if nilai < 0:
        return 'negatif'
    elif nilai == 0:
        return 'netral'
    else:
        return 'positif'

def analyze_sentiment(data_frame):
    data_frame['subjektivitas'] = data_frame['text_final'].apply(subjektivitas)
    data_frame['polaritas'] = data_frame['text_final'].apply(polaritas)
    data_frame['Sentimen'] = data_frame['polaritas'].apply(hasilSentimen)
    return data_frame

# preprocessing data text

In [ ]:
#dropnan value
start_time = time.time()
new_reviews_gi = new_gi.reset_index().dropna()
new_reviews_gi = new_reviews_gi.drop_duplicates()
new_reviews_gi['text_clean'] = new_reviews_gi['content'].apply(cleantext)#clean from number, space, symbol, and lower the text
new_reviews_gi['text_abbreviation'] = new_reviews_gi['text_clean'].apply(abbrev)#clean from slangs words
new_reviews_gi['text_lemmatize'] = new_reviews_gi['text_abbreviation'].apply(lemmatize_words)#lemmatize words
new_reviews_gi['text_tokenization'] = new_reviews_gi['text_lemmatize'].apply(tokenization)#use tokenization
new_reviews_gi['text_filtering'] = new_reviews_gi['text_tokenization'].apply(filteringsText)#filtered the text from stopwords
new_reviews_gi['text_final'] = new_reviews_gi['text_filtering'].apply(toKata)#combine token to a sentence
new_reviews_gi['text_length'] = new_reviews_gi['text_final'].apply(len)#count the length of the text
new_reviews_gi = new_reviews_gi.reset_index()
new_reviews_gi = new_reviews_gi.drop(columns=['level_0'])
print(f"Elapsed time: {time.time()-start_time}s")

In [ ]:
new_reviews_gi = new_reviews_gi.drop(columns=['index'])
new_reviews_gi.to_csv('new_reviews_gi.csv', index=False)
# !cp "/content/new_reviews_gi.csv" "/content/drive/My Drive/"#simpan ini ke google drive
new_reviews_gi

,content,score,thumbsUpCount,text_clean,text_abbreviation,text_lemmatize,text_tokenization,text_filtering,text_final,text_length
0,"Initially, I had a 5-star review. I can no lon...",3,405,initially i had a star review i can no longer...,initially i had a star review i can no longer ...,initially i have a star review i can no longer...,"[initially, i, have, a, star, review, i, can, ...","[initially, star, review, longer, keep, review...",initially star review longer keep review anymo...,291
1,This is a really great game and I do enjoy it ...,3,180,this is a really great game and i do enjoy it ...,this is a really great game and i do enjoy it ...,this be a really great game and i do enjoy it ...,"[this, be, a, really, great, game, and, i, do,...","[really, great, game, enjoy, lot, whenever, tr...",really great game enjoy lot whenever try come ...,285
2,"You've seen the adds, youtubers sponsoring thi...",5,1043,youve seen the adds youtubers sponsoring this ...,youve seen the adds youtubers sponsoring this ...,youve see the add youtubers sponsor this the w...,"[youve, see, the, add, youtubers, sponsor, thi...","[youve, see, add, youtubers, sponsor, whole, y...",youve see add youtubers sponsor whole yard tel...,301
3,For years now we have begged for a skip dialog...,1,2897,for years now we have begged for a skip dialog...,for years now we have begged for a skip dialog...,for year now we have beg for a skip dialogue b...,"[for, year, now, we, have, beg, for, a, skip, ...","[year, beg, skip, dialogue, button, even, bare...",year beg skip dialogue button even bare minimu...,308
4,Edit: I used to love this game! It has beautif...,2,987,edit i used to love this game it has beautiful...,edit i used to love this game it has beautiful...,edit i use to love this game it have beautiful...,"[edit, i, use, to, love, this, game, it, have,...","[edit, use, love, game, beautiful, music, char...",edit use love game beautiful music character s...,271
...,...,...,...,...,...,...,...,...,...,...
1610995,Very good the only thing I don't like are the ...,5,0,very good the only thing i dont like are the c...,very good the only thing i dont like are the c...,very good the only thing i dont like be the cu...,"[very, good, the, only, thing, i, dont, like, ...","[good, thing, dont, like, cut, scene]",good thing dont like cut scene,30
1610996,"The game is great overall, but the Company jus...",1,1,the game is great overall but the company just...,the game is great overall but the company just...,the game be great overall but the company just...,"[the, game, be, great, overall, but, the, comp...","[game, great, overall, company, doesnt, give, ...",game great overall company doesnt give care pe...,165
1610997,My favorite game when I'm bored it helped me a...,5,0,my favorite game when im bored it helped me a ...,my favorite game when i am bored it helped me ...,my favorite game when i be bore it help me a l...,"[my, favorite, game, when, i, be, bore, it, he...","[favorite, game, bore, help, lot, stress]",favorite game bore help lot stress,34
1610998,"This is so much fun, annoying at times, but st...",5,0,this is so much fun annoying at times but stil...,this is so much fun annoying at times but stil...,this be so much fun annoy at time but still wo...,"[this, be, so, much, fun, annoy, at, time, but...","[much, fun, annoy, time, still, worth]",much fun annoy time still worth,31


In [ ]:
del new_gi#free up memory agar tidak restart kernel di colab

## Labelling Data

In [ ]:
# translated_data_frame = translate_comments(hsr)
np.random.seed(42)
gi_label = analyze_sentiment(new_reviews_gi)
gi_label

,content,score,thumbsUpCount,text_clean,text_abbreviation,text_lemmatize,text_tokenization,text_filtering,text_final,text_length,subjektivitas,polaritas,Sentimen
0,"Initially, I had a 5-star review. I can no lon...",3,405,initially i had a star review i can no longer...,initially i had a star review i can no longer ...,initially i have a star review i can no longer...,"[initially, i, have, a, star, review, i, can, ...","[initially, star, review, longer, keep, review...",initially star review longer keep review anymo...,291,0.493939,0.242541,positif
1,This is a really great game and I do enjoy it ...,3,180,this is a really great game and i do enjoy it ...,this is a really great game and i do enjoy it ...,this be a really great game and i do enjoy it ...,"[this, be, a, really, great, game, and, i, do,...","[really, great, game, enjoy, lot, whenever, tr...",really great game enjoy lot whenever try come ...,285,0.405556,0.100000,positif
2,"You've seen the adds, youtubers sponsoring thi...",5,1043,youve seen the adds youtubers sponsoring this ...,youve seen the adds youtubers sponsoring this ...,youve see the add youtubers sponsor this the w...,"[youve, see, the, add, youtubers, sponsor, thi...","[youve, see, add, youtubers, sponsor, whole, y...",youve see add youtubers sponsor whole yard tel...,301,0.531429,-0.013571,negatif
3,For years now we have begged for a skip dialog...,1,2897,for years now we have begged for a skip dialog...,for years now we have begged for a skip dialog...,for year now we have beg for a skip dialogue b...,"[for, year, now, we, have, beg, for, a, skip, ...","[year, beg, skip, dialogue, button, even, bare...",year beg skip dialogue button even bare minimu...,308,0.270000,-0.058333,negatif
4,Edit: I used to love this game! It has beautif...,2,987,edit i used to love this game it has beautiful...,edit i used to love this game it has beautiful...,edit i use to love this game it have beautiful...,"[edit, i, use, to, love, this, game, it, have,...","[edit, use, love, game, beautiful, music, char...",edit use love game beautiful music character s...,271,0.543333,-0.078333,negatif
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1610995,Very good the only thing I don't like are the ...,5,0,very good the only thing i dont like are the c...,very good the only thing i dont like are the c...,very good the only thing i dont like be the cu...,"[very, good, the, only, thing, i, dont, like, ...","[good, thing, dont, like, cut, scene]",good thing dont like cut scene,30,0.600000,0.700000,positif
1610996,"The game is great overall, but the Company jus...",1,1,the game is great overall but the company just...,the game is great overall but the company just...,the game be great overall but the company just...,"[the, game, be, great, overall, but, the, comp...","[game, great, overall, company, doesnt, give, ...",game great overall company doesnt give care pe...,165,0.390000,-0.080000,negatif
1610997,My favorite game when I'm bored it helped me a...,5,0,my favorite game when im bored it helped me a ...,my favorite game when i am bored it helped me ...,my favorite game when i be bore it help me a l...,"[my, favorite, game, when, i, be, bore, it, he...","[favorite, game, bore, help, lot, stress]",favorite game bore help lot stress,34,0.700000,0.050000,positif
1610998,"This is so much fun, annoying at times, but st...",5,0,this is so much fun annoying at times but stil...,this is so much fun annoying at times but stil...,this be so much fun annoy at time but still wo...,"[this, be, so, much, fun, annoy, at, time, but...","[much, fun, annoy, time, still, worth]",much fun annoy time still worth,31,0.150000,0.300000,positif


In [ ]:
print(gi_label['Sentimen'].value_counts())

Sentimen
positif    1031132
negatif     499646
netral       80222
Name: count, dtype: int64


In [ ]:
print(gi_label.isna().sum())
print(gi_label.describe())

content              0
score                0
thumbsUpCount        0
text_clean           0
text_abbreviation    0
text_lemmatize       0
text_tokenization    0
text_filtering       0
text_final           0
text_length          0
subjektivitas        0
polaritas            0
Sentimen             0
dtype: int64
              score  thumbsUpCount   text_length  subjektivitas     polaritas
count  1.611000e+06   1.611000e+06  1.611000e+06   1.611000e+06  1.611000e+06
mean   3.709377e+00   5.397897e+00  1.184942e+02   4.958091e-01  8.412640e-02
std    1.597130e+00   5.594445e+01  8.862254e+01   1.643540e-01  2.470005e-01
min    1.000000e+00   0.000000e+00  0.000000e+00   0.000000e+00 -1.000000e+00
25%    2.000000e+00   0.000000e+00  5.500000e+01   4.059524e-01 -5.000000e-02
50%    5.000000e+00   0.000000e+00  9.000000e+01   5.000000e-01  8.727273e-02
75%    5.000000e+00   1.000000e+00  1.600000e+02   5.925000e-01  2.300595e-01
max    5.000000e+00   5.620000e+03  2.450000e+03   1.000000e+00 

In [ ]:
gi_label = gi_label.drop(columns=['score','thumbsUpCount'])
gi_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1611000 entries, 0 to 1610999
Data columns (total 11 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   content            1611000 non-null  object 
 1   text_clean         1611000 non-null  object 
 2   text_abbreviation  1611000 non-null  object 
 3   text_lemmatize     1611000 non-null  object 
 4   text_tokenization  1611000 non-null  object 
 5   text_filtering     1611000 non-null  object 
 6   text_final         1611000 non-null  object 
 7   text_length        1611000 non-null  int64  
 8   subjektivitas      1611000 non-null  float64
 9   polaritas          1611000 non-null  float64
 10  Sentimen           1611000 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 135.2+ MB


In [ ]:
gi_label.to_csv('gi_label.csv', index=False)

In [ ]:
# !cp "/content/gi_label.csv" "/content/drive/My Drive/"#uncomment untuk save data ke CSV